# Exercise 6

## SVM & Regularization


For this homework we consider a set of observations on a number of red and white wine varieties involving their chemical properties and ranking by tasters. Wine industry shows a recent growth spurt as social drinking is on the rise. The price of wine depends on a rather abstract concept of wine appreciation by wine tasters, opinion among whom may have a high degree of variability. Pricing of wine depends on such a volatile factor to some extent. Another key factor in wine certification and quality assessment is physicochemical tests which are laboratory-based and takes into account factors like acidity, pH level, presence of sugar and other chemical properties. For the wine market, it would be of interest if human quality of tasting can be related to the chemical properties of wine so that certification and quality assessment and assurance process is more controlled.

Two datasets are available of which one dataset is on red wine and have 1599 different varieties and the other is on white wine and have 4898 varieties. All wines are produced in a particular area of Portugal. Data are collected on 12 different properties of the wines one of which is Quality, based on sensory data, and the rest are on chemical properties of the wines including density, acidity, alcohol content etc. All chemical properties of wines are continuous variables. Quality is an ordinal variable with possible ranking from 1 (worst) to 10 (best). Each variety of wine is tasted by three independent tasters and the final rank assigned is the median rank given by the tasters.

A predictive model developed on this data is expected to provide guidance to vineyards regarding quality and price expected on their produce without heavy reliance on volatility of wine tasters.

In [1]:
%connect_info

{
  "shell_port": 60466,
  "iopub_port": 60467,
  "stdin_port": 60468,
  "control_port": 60469,
  "hb_port": 60470,
  "ip": "127.0.0.1",
  "key": "90e5acef-a2ce7b25be4f3e86d93986a8",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-d9eb96f0-f1d6-4828-899c-b6de9d3630b2.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [1]:
import pandas as pd
import numpy as np

In [2]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [3]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
4841,5.7,0.22,0.25,1.1,0.050,97.0,175.0,0.99099,3.44,0.62,11.1,6,white
1444,7.8,0.43,0.49,13.0,0.033,37.0,158.0,0.99550,3.14,0.35,11.3,6,white
1227,6.9,0.18,0.38,6.5,0.039,20.0,110.0,0.99430,3.10,0.42,10.5,5,white
2438,6.9,0.43,0.28,9.4,0.056,29.0,183.0,0.99594,3.17,0.43,9.4,5,white
4500,7.8,0.27,0.33,2.4,0.053,36.0,175.0,0.99200,3.20,0.55,11.0,6,white


# Exercise 6.1

Show the frecuency table of the quality by type of wine

Se busca clasificar cuales son buenos y malos en binario

In [23]:
data[['quality','type']].groupby('quality').count()

,type
quality,
3,30
4,216
5,2138
6,2836
7,1079
8,193
9,5


In [24]:
data['quality2'] = data['quality'].map({'3':0,'4':0,'5':0,'6':0,'7':1,'8':1,'9':1})

In [25]:
data_white=data[data['type']=='white']
data_red=data[data['type']=='red']
data_white.shape,data_red.shape

((4898, 14), (1599, 14))

# SVM

# Exercise 6.2

* Standarized the features (not the quality)
* Create a binary target for each type of wine
* Create two Linear SVM's for the white and red wines, repectively.


Se crean dos modelos de clasificación, uno por cada tipo de vino, con el fin de evaluar la calidad de cada tipo de vino.
Una forma de estandarizar es normalizar, buscar otras formas de estandarizar, usar preproccesing.normalize

# Exercise 6.3

Test the two SVM's using the different kernels (‘poly’, ‘rbf’, ‘sigmoid’)


# Exercise 6.4
Using the best SVM find the parameters that gives the best performance

'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]

Se evalua con la clasificación: el accuracy

# Exercise 6.5

Compare the results with other methods

Se compara con LOGIT

# Regularization

# Exercise 6.6


* Train a linear regression to predict wine quality (Continous) (Lo mismo del 6.5)

* Analyze the coefficients

* Evaluate the RMSE

# Exercise 6.7

* Estimate a ridge regression with alpha equals 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

# Exercise 6.8

* Estimate a lasso regression with alpha equals 0.01, 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

# Exercise 6.9

* Create a binary target

* Train a logistic regression to predict wine quality (binary)

* Analyze the coefficients

* Evaluate the f1score

# Exercise 6.10

* Estimate a regularized logistic regression using:
* C = 0.01, 0.1 & 1.0
* penalty = ['l1, 'l2']
* Compare the coefficients and the f1score

Usando regularización